In [4]:
def compute_equiv_q(M1, M2, q):
    """
    Compute all sigma in GL_n(F_q) such that sigma * M1 * sigma^(-1) = M2
    
    Parameters:
    -----------
    M1, M2 : matrices (will be converted to F_q)
    q : prime power
    
    Returns:
    --------
    List of matrices sigma in GL_n(F_q) satisfying the equation
    """
    F = GF(q)
    n = M1.nrows()
    
    # Convert to matrices over F_q
    M1_q = matrix(F, M1)
    M2_q = matrix(F, M2)
    
    # Find all solutions
    solutions = []
    for sigma in GL(n, F):
        if sigma * M1_q * sigma^(-1) == M2_q:
            solutions.append(sigma)
    
    return solutions

In [5]:
def is_permutation_matrix(sigma):
    """
    Check if sigma is a permutation matrix (entries in {0,1}, one 1 per row/col)
    """
    n = sigma.matrix().nrows()
    F = sigma.base_ring()
    
    # Check entries are 0 or 1
    for i in range(n):
        for j in range(n):
            if sigma.matrix()[i,j] not in [F(0), F(1)]:
                return False
    
    # Check exactly one 1 per row
    for i in range(n):
        if sum(sigma.matrix()[i,j] for j in range(n)) != F(1):
            return False
    
    # Check exactly one 1 per column
    for j in range(n):
        if sum(sigma.matrix()[i,j] for i in range(n)) != F(1):
            return False
    
    return True

In [6]:
# Example usage for n=3, q=2
F = GF(2)

# Path graph P_3: 1-2-3
M1 = matrix(F, [[0,1,0], [1,0,1], [0,1,0]])

# Same path, relabeled: 1-3-2
M2 = matrix(F, [[0,0,1], [0,0,1], [1,1,0]])

print("M1 (Path 1-2-3):")
print(M1)
print("\nM2 (Path 1-3-2):")
print(M2)

solutions = compute_equiv_q(M1, M2, 2)
print(f"\nFound {len(solutions)} solutions in GL_3(F_2)")
print(f"|GL_3(F_2)| = {GL(3, F).order()}")

perm_solutions = [s for s in solutions if is_permutation_matrix(s)]
print(f"\n{len(perm_solutions)} are permutation matrices")
print(f"{len(solutions) - len(perm_solutions)} are non-permutation matrices")

print("\nPermutation solutions:")
for i, sigma in enumerate(perm_solutions):
    print(f"\nPermutation {i+1}:")
    print(sigma)

print("\nNon-permutation solutions:")
for i, sigma in enumerate(solutions):
    if not is_permutation_matrix(sigma):
        print(f"\nNon-permutation {i+1}:")
        print(sigma)
        print(f"Determinant: {sigma.matrix().det()}")

# Verify one solution works
if solutions:
    sigma = solutions[0]
    result = sigma * M1 * sigma^(-1)
    print(f"\nVerification: sigma * M1 * sigma^(-1) == M2: {result == M2}")

M1 (Path 1-2-3):
[0 1 0]
[1 0 1]
[0 1 0]

M2 (Path 1-3-2):
[0 0 1]
[0 0 1]
[1 1 0]

Found 4 solutions in GL_3(F_2)
|GL_3(F_2)| = 168

2 are permutation matrices
2 are non-permutation matrices

Permutation solutions:

Permutation 1:
[0 0 1]
[1 0 0]
[0 1 0]

Permutation 2:
[1 0 0]
[0 0 1]
[0 1 0]

Non-permutation solutions:

Non-permutation 2:
[0 1 1]
[1 1 0]
[1 1 1]
Determinant: 1

Non-permutation 4:
[1 1 0]
[0 1 1]
[1 1 1]
Determinant: 1

Verification: sigma * M1 * sigma^(-1) == M2: True


In [17]:
def are_similar_fast(M1, M2, q):
    """
    Fast check if M1 and M2 are similar over F_q using canonical forms.
    Returns True/False without finding all solutions.
    """
    F = GF(q)
    M1_q = matrix(F, M1)
    M2_q = matrix(F, M2)
    
    # Check if they have the same rational canonical form
    # or characteristic polynomial as a quick test
    if M1_q.charpoly() != M2_q.charpoly():
        return False
    
    # For definitive answer, compare rational canonical forms
    try:
        # In Sage, this computes the rational canonical form
        rcf1 = M1_q.rational_form()
        rcf2 = M2_q.rational_form()
        return rcf1 == rcf2
    except:
        # Fallback: just use characteristic polynomial
        return M1_q.charpoly() == M2_q.charpoly()

In [8]:
are_similar_fast(M1,M2,2)

True

In [18]:
def find_one_solution(M1, M2, q):
    """
    If M1 and M2 are similar, find ONE solution sigma.
    This is much faster than enumerating all solutions.
    """
    F = GF(q)
    M1_q = matrix(F, M1)
    M2_q = matrix(F, M2)
    
    # Get rational canonical forms and transformation matrices
    rcf1, P1 = M1_q.rational_form(transformation=True)
    rcf2, P2 = M2_q.rational_form(transformation=True)
    
    if rcf1 != rcf2:
        return None  # Not similar
    
    # P1 * M1 * P1^(-1) = rcf = P2 * M2 * P2^(-1)
    # So: M2 = P2^(-1) * rcf * P2 = P2^(-1) * P1 * M1 * P1^(-1) * P2
    # Therefore: sigma = P2^(-1) * P1 satisfies sigma * M1 * sigma^(-1) = M2
    sigma = P2^(-1) * P1
    return sigma

In [16]:

# `rational_form()` does not provide the change-of-basis matrix.
find_one_solution(M1,M2,2)

TypeError: rational_form() got an unexpected keyword argument 'transformation'